OLS in simple regression

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
from scipy import stats
import statsmodels.api as sm

# 1) Generate synthetic data
# ----------------------------
rng = np.random.default_rng(2025)
n = 1200
beta0_true, beta1_true = 0.02, 0.75
X = rng.normal(0.0, 1.2, n)
eps = rng.normal(0.0, 0.45, n)
Y = beta0_true + beta1_true * X + eps

df = pd.DataFrame({"X": X, "Y": Y})

# 2) Closed-form OLS estimators
# ----------------------------
xbar, ybar = df["X"].mean(), df["Y"].mean()
Sxx = float(((df["X"] - xbar)**2).sum())
Sxy = float(((df["X"] - xbar)*(df["Y"] - ybar)).sum())

beta1_hat = Sxy / Sxx
beta0_hat = ybar - beta1_hat * xbar

# Fitted values, residuals, variance estimate
Y_hat = beta0_hat + beta1_hat * df["X"].to_numpy()
resid = df["Y"].to_numpy() - Y_hat
df_resid = n - 2
sigma2_hat = (resid @ resid) / df_resid

# Standard errors
se_beta1 = np.sqrt(sigma2_hat / Sxx)
se_beta0 = np.sqrt(sigma2_hat * (1.0/n + (xbar**2)/Sxx))

# t-stats, two-sided 95% CIs
t_beta1 = beta1_hat / se_beta1
t_beta0 = beta0_hat / se_beta0
tcrit = stats.t.ppf(0.975, df_resid)
ci_beta1 = (beta1_hat - tcrit*se_beta1, beta1_hat + tcrit*se_beta1)
ci_beta0 = (beta0_hat - tcrit*se_beta0, beta0_hat + tcrit*se_beta0)

# Goodness of fit
TSS = float(((df["Y"] - ybar)**2).sum())
RSS = float((resid**2).sum())
R2 = 1.0 - RSS / TSS
adj_R2 = 1.0 - (1.0 - R2) * (n - 1) / (n - 2)

# 3) statsmodels cross-check
# ----------------------------
X1 = sm.add_constant(df["X"].to_numpy(), has_constant="add")
ols = sm.OLS(df["Y"].to_numpy(), X1).fit()
ols_hc1 = ols.get_robustcov_results(cov_type="HC1")

# 4) Mean-response CI and prediction interval at x0
#    mean CI: yhat ± tcrit * sqrt( sigma^2 * [ 1/n + (x0 - xbar)^2 / Sxx ] )
#    pred PI: yhat ± tcrit * sqrt( sigma^2 * [ 1 + 1/n + (x0 - xbar)^2 / Sxx ] )
# ----------------------------
x0 = 1.00
y0_hat = beta0_hat + beta1_hat * x0
se_mean = np.sqrt(sigma2_hat * (1.0/n + ((x0 - xbar)**2)/Sxx))
se_pred = np.sqrt(sigma2_hat * (1.0 + 1.0/n + ((x0 - xbar)**2)/Sxx))
ci_mean = (y0_hat - tcrit*se_mean, y0_hat + tcrit*se_mean)
pi_pred = (y0_hat - tcrit*se_pred, y0_hat + tcrit*se_pred)

# statsmodels check via get_prediction
pred = ols.get_prediction(exog=[1.0, x0])
ci_mean_sm = tuple(pred.summary_frame(alpha=0.05)[["mean_ci_lower","mean_ci_upper"]].iloc[0])
pi_pred_sm = tuple(pred.summary_frame(alpha=0.05)[["obs_ci_lower","obs_ci_upper"]].iloc[0])

print("="*72)
print("Simple Linear Regression — OLS (closed-form) vs statsmodels")
print("="*72)
print(f"True betas      : beta0={beta0_true:.3f}, beta1={beta1_true:.3f}")
print(f"Closed-form OLS : beta0={beta0_hat:.3f} (se={se_beta0:.3f}, t={beta0_hat/se_beta0:.2f}, CI95={ci_beta0})")
print(f"                  beta1={beta1_hat:.3f} (se={se_beta1:.3f}, t={t_beta1:.2f}, CI95={ci_beta1})")
print(f"Fit             : R^2={R2:.4f}, adj R^2={adj_R2:.4f}")
print("-"*72)
print("statsmodels OLS (homoskedastic):")
print(f"  beta0={ols.params[0]:.3f} (se={ols.bse[0]:.3f}), "
      f"beta1={ols.params[1]:.3f} (se={ols.bse[1]:.3f}), R^2={ols.rsquared:.4f}")
print("statsmodels OLS (HC1 robust SE):")
print(f"  beta0={ols_hc1.params[0]:.3f} (se={ols_hc1.bse[0]:.3f}), "
      f"beta1={ols_hc1.params[1]:.3f} (se={ols_hc1.bse[1]:.3f})")
print("-"*72)
print(f"Mean-response CI at x0={x0:.2f} : {ci_mean} (closed-form) vs {ci_mean_sm} (sm)")
print(f"Prediction interval at x0={x0:.2f}: {pi_pred} (closed-form) vs {pi_pred_sm} (sm)")
print("="*72)

Simple Linear Regression — OLS (closed-form) vs statsmodels
True betas      : beta0=0.020, beta1=0.750
Closed-form OLS : beta0=0.003 (se=0.013, t=0.25, CI95=(np.float64(-0.022367787047077002), np.float64(0.028988837442635945)))
                  beta1=0.758 (se=0.011, t=69.30, CI95=(np.float64(0.736394877958875), np.float64(0.7793051122689277)))
Fit             : R^2=0.8004, adj R^2=0.8002
------------------------------------------------------------------------
statsmodels OLS (homoskedastic):
  beta0=0.003 (se=0.013), beta1=0.758 (se=0.011), R^2=0.8004
statsmodels OLS (HC1 robust SE):
  beta0=0.003 (se=0.013), beta1=0.758 (se=0.011)
------------------------------------------------------------------------
Mean-response CI at x0=1.00 : (np.float64(0.7273899445498223), np.float64(0.7949310960735395)) (closed-form) vs (0.727389944549822, 0.7949310960735392) (sm)
Prediction interval at x0=1.00: (np.float64(-0.12884543567577822), np.float64(1.65116647629914)) (closed-form) vs (-0.1288454356

Confidence intervals of slope and intercept

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
from scipy import stats
import statsmodels.api as sm


# 1) Synthetic data
# ----------------------------
rng = np.random.default_rng(101)
n = 1000
beta0_true, beta1_true = 0.02, 0.80
X = rng.normal(0.0, 1.0, n)
# Heteroskedastic errors: variance increases with |X|
sigma = 0.30 * (1.0 + 0.7 * np.abs(X))
eps = rng.normal(0.0, sigma, n)
Y = beta0_true + beta1_true * X + eps

df = pd.DataFrame({"X": X, "Y": Y})


# 2) Classical OLS (closed-form) and CIs
# ----------------------------
xbar, ybar = df["X"].mean(), df["Y"].mean()
Sxx = float(((df["X"] - xbar)**2).sum())
Sxy = float(((df["X"] - xbar)*(df["Y"] - ybar)).sum())

beta1_hat = Sxy / Sxx
beta0_hat = ybar - beta1_hat * xbar

# Residual variance (homoskedastic OLS)
Y_hat = beta0_hat + beta1_hat * df["X"].to_numpy()
resid = df["Y"].to_numpy() - Y_hat
df_resid = n - 2
sigma2_hat = (resid @ resid) / df_resid

# Standard errors
se_beta1 = np.sqrt(sigma2_hat / Sxx)
se_beta0 = np.sqrt(sigma2_hat * (1.0/n + (xbar**2)/Sxx))

# 95% CIs using Student-t with (n-2) dof
tcrit = stats.t.ppf(0.975, df_resid)
ci_beta1_classical = (beta1_hat - tcrit*se_beta1, beta1_hat + tcrit*se_beta1)
ci_beta0_classical = (beta0_hat - tcrit*se_beta0, beta0_hat + tcrit*se_beta0)


# 3) HC1-robust CIs via statsmodels
# ----------------------------
X1 = sm.add_constant(df["X"].to_numpy(), has_constant="add")
ols = sm.OLS(df["Y"].to_numpy(), X1).fit()
ols_hc1 = ols.get_robustcov_results(cov_type="HC1")  # heteroskedasticity-robust

beta0_sm, beta1_sm = float(ols_hc1.params[0]), float(ols_hc1.params[1])
se_beta0_hc1, se_beta1_hc1 = float(ols_hc1.bse[0]), float(ols_hc1.bse[1])

# Use the same t critical (large-sample normal would give a similar result)
ci_beta1_hc1 = (beta1_sm - tcrit*se_beta1_hc1, beta1_sm + tcrit*se_beta1_hc1)
ci_beta0_hc1 = (beta0_sm - tcrit*se_beta0_hc1, beta0_sm + tcrit*se_beta0_hc1)

def fmt(ci): return f"({ci[0]:.3f}, {ci[1]:.3f})"

print("="*72)
print("Confidence Intervals for Simple Linear Regression Coefficients")
print("="*72)
print(f"True betas: beta0={beta0_true:.3f}, beta1={beta1_true:.3f}\n")

print("Classical OLS (homoskedastic) estimates & 95% CIs:")
print(f"  beta0_hat={beta0_hat:.3f}, CI95={fmt(ci_beta0_classical)}")
print(f"  beta1_hat={beta1_hat:.3f}, CI95={fmt(ci_beta1_classical)}\n")

print("HC1-robust (heteroskedasticity-robust) estimates & 95% CIs:")
print(f"  beta0_sm ={beta0_sm:.3f},  CI95={fmt(ci_beta0_hc1)}")
print(f"  beta1_sm ={beta1_sm:.3f},  CI95={fmt(ci_beta1_hc1)}")
print("="*72)

Confidence Intervals for Simple Linear Regression Coefficients
True betas: beta0=0.020, beta1=0.800

Classical OLS (homoskedastic) estimates & 95% CIs:
  beta0_hat=0.013, CI95=(-0.017, 0.044)
  beta1_hat=0.842, CI95=(0.813, 0.871)

HC1-robust (heteroskedasticity-robust) estimates & 95% CIs:
  beta0_sm =0.013,  CI95=(-0.017, 0.044)
  beta1_sm =0.842,  CI95=(0.804, 0.880)


Confidence interval of the mean response

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
from scipy import stats
import statsmodels.api as sm

# 1) Synthetic data
# ----------------------------
rng = np.random.default_rng(314)
n = 1000
beta0_true, beta1_true = 0.02, 0.80
X = rng.normal(0.0, 1.0, n)
# Heteroskedastic errors: variance grows with |X|
sigma = 0.35 * (1.0 + 0.7 * np.abs(X))
eps = rng.normal(0.0, sigma, n)
Y = beta0_true + beta1_true * X + eps

df = pd.DataFrame({"X": X, "Y": Y})


# 2) Closed-form OLS estimates
# ----------------------------
xbar, ybar = df["X"].mean(), df["Y"].mean()
Sxx = float(((df["X"] - xbar)**2).sum())
Sxy = float(((df["X"] - xbar)*(df["Y"] - ybar)).sum())

beta1_hat = Sxy / Sxx
beta0_hat = ybar - beta1_hat * xbar

Y_hat = beta0_hat + beta1_hat * df["X"].to_numpy()
resid = df["Y"].to_numpy() - Y_hat
df_resid = n - 2
sigma2_hat = (resid @ resid) / df_resid
tcrit = stats.t.ppf(0.975, df_resid)  # 95% two-sided


# 3) HC1-robust covariance for parameters
# ----------------------------
X1 = sm.add_constant(df["X"].to_numpy(), has_constant="add")
ols = sm.OLS(df["Y"].to_numpy(), X1).fit()
ols_hc1 = ols.get_robustcov_results(cov_type="HC1")
Cov_hc1 = ols_hc1.cov_params()  # 2x2 covariance of [beta0, beta1]


# 4) CI for mean response at selected X0 values
# Classical: Var(ŷ(x0)) = σ^2 * [ 1/n + (x0 - xbar)^2 / Sxx ]
# Robust:    Var(ŷ(x0)) = x0_vec' * Cov(beta) * x0_vec,  x0_vec=[1, x0]
# ----------------------------
def ci_mean_classical(x0: float):
    yhat = beta0_hat + beta1_hat * x0
    se = np.sqrt(sigma2_hat * (1.0/n + ((x0 - xbar)**2)/Sxx))
    return yhat, (yhat - tcrit*se, yhat + tcrit*se), se

def ci_mean_robust(x0: float):
    xvec = np.array([1.0, x0])
    yhat = float(np.dot(xvec, np.array([ols_hc1.params[0], ols_hc1.params[1]])))
    se = float(np.sqrt(xvec @ Cov_hc1 @ xvec))
    return yhat, (yhat - tcrit*se, yhat + tcrit*se), se

# Choose points: below mean, at mean, above mean
x0_list = [np.quantile(X, 0.15), xbar, np.quantile(X, 0.85)]

print("="*72)
print("Confidence intervals for the MEAN response E[Y|X0]")
print("="*72)
for x0 in x0_list:
    y_c, ci_c, se_c = ci_mean_classical(float(x0))
    y_r, ci_r, se_r = ci_mean_robust(float(x0))
    print(f"X0 = {x0: .3f}")
    print(f"  Classical ŷ={y_c: .3f},  CI95={ci_c},  SE={se_c:.4f}")
    print(f"  Robust    ŷ={y_r: .3f},  CI95={ci_r},  SE={se_r:.4f}")
    print("-"*72)

Confidence intervals for the MEAN response E[Y|X0]
X0 = -0.895
  Classical ŷ=-0.735,  CI95=(np.float64(-0.7825684980182602), np.float64(-0.6873777229876489)),  SE=0.0243
  Robust    ŷ=-0.735,  CI95=(np.float64(-0.7918742713952023), np.float64(-0.6780719496107086)),  SE=0.0290
------------------------------------------------------------------------
X0 =  0.037
  Classical ŷ= 0.019,  CI95=(np.float64(-0.015854160115375375), np.float64(0.05302095371959762)),  SE=0.0175
  Robust    ŷ= 0.019,  CI95=(np.float64(-0.015854160115375444), np.float64(0.05302095371959758)),  SE=0.0175
------------------------------------------------------------------------
X0 =  1.038
  Classical ŷ= 0.828,  CI95=(np.float64(0.7785160881547518), np.float64(0.8771207257613852)),  SE=0.0251
  Robust    ŷ= 0.828,  CI95=(np.float64(0.7667670579810026), np.float64(0.888869755935136)),  SE=0.0311
------------------------------------------------------------------------


Prediction interval of a new observation

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
from scipy import stats
import statsmodels.api as sm


# 1) Synthetic data
# ----------------------------
rng = np.random.default_rng(2718)
n = 1000
beta0_true, beta1_true = 0.02, 0.80
X = rng.normal(0.0, 1.0, n)
# Heteroskedastic errors: variance increases with |X|
sigma = 0.35 * (1.0 + 0.7 * np.abs(X))
eps = rng.normal(0.0, sigma, n)
Y = beta0_true + beta1_true * X + eps

df = pd.DataFrame({"X": X, "Y": Y})


# 2) Closed-form OLS estimates
# ----------------------------
xbar, ybar = df["X"].mean(), df["Y"].mean()
Sxx = float(((df["X"] - xbar)**2).sum())
Sxy = float(((df["X"] - xbar)*(df["Y"] - ybar)).sum())

beta1_hat = Sxy / Sxx
beta0_hat = ybar - beta1_hat * xbar

Y_hat = beta0_hat + beta1_hat * df["X"].to_numpy()
resid = df["Y"].to_numpy() - Y_hat
df_resid = n - 2
sigma2_hat = (resid @ resid) / df_resid
tcrit = stats.t.ppf(0.975, df_resid)  # 95% two-sided

# 3) HC1-robust covariance for parameters
# ----------------------------
X1 = sm.add_constant(df["X"].to_numpy(), has_constant="add")
ols = sm.OLS(df["Y"].to_numpy(), X1).fit()
ols_hc1 = ols.get_robustcov_results(cov_type="HC1")
Cov_hc1 = ols_hc1.cov_params()  # 2x2 covariance of [beta0, beta1]

# 4) Prediction interval at selected X0 values
# Classical PI variance:
#   Var_pred_classical(x0) = σ^2 * [ 1 + 1/n + (x0 - xbar)^2 / Sxx ]
# Robust PI variance (large-sample heuristic):
#   Var_pred_robust(x0) = x0_vec' Cov_HC1(beta) x0_vec + σ^2
# where x0_vec = [1, x0]
# ----------------------------
def pi_classical(x0: float):
    yhat = beta0_hat + beta1_hat * x0
    se_pred = np.sqrt(sigma2_hat * (1.0 + 1.0/n + ((x0 - xbar)**2)/Sxx))
    return yhat, (yhat - tcrit*se_pred, yhat + tcrit*se_pred), se_pred

def pi_robust(x0: float):
    xvec = np.array([1.0, x0])
    yhat = float(xvec @ np.array([ols_hc1.params[0], ols_hc1.params[1]]))
    var_pred = float(xvec @ Cov_hc1 @ xvec) + sigma2_hat
    se_pred = np.sqrt(var_pred)
    return yhat, (yhat - tcrit*se_pred, yhat + tcrit*se_pred), se_pred

# Optional sm cross-check for classical PI at a single point
def pi_sm(x0: float):
    fr = ols.get_prediction(exog=[1.0, x0]).summary_frame(alpha=0.05)
    return float(fr["obs_ci_lower"][0]), float(fr["obs_ci_upper"][0])

# Choose points: below mean, at mean, above mean
x0_list = [np.quantile(X, 0.15), xbar, np.quantile(X, 0.85)]

print("="*72)
print("Prediction intervals for a NEW observation Y_new | X0")
print("="*72)
for x0 in x0_list:
    y_c, pi_c, se_c = pi_classical(float(x0))
    y_r, pi_r, se_r = pi_robust(float(x0))
    pi_sm_lo, pi_sm_hi = pi_sm(float(x0))
    print(f"X0 = {x0: .3f}")
    print(f"  Classical ŷ={y_c: .3f},  PI95={pi_c},  SE_pred={se_c:.4f}  [sm check: ({pi_sm_lo:.3f}, {pi_sm_hi:.3f})]")
    print(f"  Robust    ŷ={y_r: .3f},  PI95={pi_r},  SE_pred={se_r:.4f}")
    print("-"*72)

Prediction intervals for a NEW observation Y_new | X0
X0 = -1.114
  Classical ŷ=-0.863,  PI95=(np.float64(-2.0011642082059193), np.float64(0.2752796328380316)),  SE_pred=0.5800  [sm check: (-2.001, 0.275)]
  Robust    ŷ=-0.863,  PI95=(np.float64(-2.0018114455082334), np.float64(0.275926870140346)),  SE_pred=0.5804
------------------------------------------------------------------------
X0 = -0.023
  Classical ŷ= 0.016,  PI95=(np.float64(-1.121748792540356), np.float64(1.1534928169165177)),  SE_pred=0.5797  [sm check: (-1.122, 1.153)]
  Robust    ŷ= 0.016,  PI95=(np.float64(-1.1217487925403558), np.float64(1.153492816916518)),  SE_pred=0.5797
------------------------------------------------------------------------
X0 =  1.032
  Classical ŷ= 0.866,  PI95=(np.float64(-0.27242677352793354), np.float64(2.003939235990842)),  SE_pred=0.5800  [sm check: (-0.272, 2.004)]
  Robust    ŷ= 0.866,  PI95=(np.float64(-0.27289336674327636), np.float64(2.004405829206185)),  SE_pred=0.5802
--------------

Coefficient of determination

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(123)

def r2_metrics(y_true: np.ndarray, y_hat: np.ndarray, n_params: int) -> dict:
    """Compute SSE, SST, R^2, adjusted R^2 given y and fitted values."""
    y_true = np.asarray(y_true); y_hat = np.asarray(y_hat)
    sse = float(np.sum((y_true - y_hat)**2))
    sst = float(np.sum((y_true - y_true.mean())**2))
    r2 = 1.0 - sse / sst
    n = y_true.size
    k = n_params  # number of slope parameters (exclude intercept)
    adj_r2 = 1.0 - (1.0 - r2) * (n - 1) / (n - 1 - k)
    return {"SSE": sse, "SST": sst, "R2": r2, "adj_R2": adj_r2}

def fit_linear(X: np.ndarray, y: np.ndarray):
    X1 = sm.add_constant(X, has_constant="add")
    model = sm.OLS(y, X1).fit()
    yhat = model.fittedvalues
    # n_params = number of slopes (exclude intercept) -> 1 here
    return model, np.asarray(yhat), 1

def fit_quadratic(X: np.ndarray, y: np.ndarray):
    X2 = np.column_stack([X, X**2])
    X2 = sm.add_constant(X2, has_constant="add")
    model = sm.OLS(y, X2).fit()
    yhat = model.fittedvalues
    # two slopes: X and X^2
    return model, np.asarray(yhat), 2

def oos_r2(model, X_test: np.ndarray, y_test: np.ndarray) -> float:
    """Out-of-sample R^2 = 1 - SSE_test / SST_test (baseline is train-mean of y)."""
    # Build the appropriate design matrix for prediction based on the model exog shape
    if model.model.exog.shape[1] == 2:  # const + X
        Xo = sm.add_constant(X_test, has_constant="add")
    else:  # const + X + X^2
        Xo = sm.add_constant(np.column_stack([X_test, X_test**2]), has_constant="add")
    yhat = model.predict(Xo)
    sse = float(np.sum((y_test - yhat)**2))
    sst = float(np.sum((y_test - y_test.mean())**2))
    return 1.0 - sse / sst

# 1) Linear DGP
# ----------------------------
n = 1500
X_lin = rng.normal(0.0, 1.0, n)
eps_lin = rng.normal(0.0, 0.5, n)
beta0_L, beta1_L = 0.5, 1.2
Y_lin = beta0_L + beta1_L * X_lin + eps_lin

# Fit simple linear model
mL, yhatL, kL = fit_linear(X_lin, Y_lin)
metL = r2_metrics(Y_lin, yhatL, n_params=kL)

# Train/Test split and OOS R^2
idx = rng.permutation(n)
train = idx[: int(0.7*n)]
test  = idx[int(0.7*n):]
r2_oos_L = oos_r2(mL, X_lin[test], Y_lin[test])

print("="*72)
print("Coefficient of Determination — Linear DGP")
print("="*72)
print(f"In-sample: R^2 = {metL['R2']:.4f},  adj R^2 = {metL['adj_R2']:.4f}")
print(f"Out-of-sample R^2 (test): {r2_oos_L:.4f}")
print("-"*72)

# 2) Nonlinear DGP (quadratic component)
# ----------------------------
X_nl = rng.normal(0.0, 1.2, n)
eps_nl = rng.normal(0.0, 0.6, n)
beta0_NL, beta1_NL, beta2_NL = 0.2, 0.3, 0.9
Y_nl = beta0_NL + beta1_NL * X_nl + beta2_NL * (X_nl**2) + eps_nl

# (a) Fit simple linear model (misspecified)
mN_lin, yhatN_lin, kN_lin = fit_linear(X_nl, Y_nl)
metN_lin = r2_metrics(Y_nl, yhatN_lin, n_params=kN_lin)

# (b) Fit transformed model with quadratic term
mN_quad, yhatN_quad, kN_quad = fit_quadratic(X_nl, Y_nl)
metN_quad = r2_metrics(Y_nl, yhatN_quad, n_params=kN_quad)

# OOS R^2 for both (same split logic)
idx2 = rng.permutation(n)
train2 = idx2[: int(0.7*n)]
test2  = idx2[int(0.7*n):]
r2_oos_N_lin  = oos_r2(mN_lin,  X_nl[test2], Y_nl[test2])
r2_oos_N_quad = oos_r2(mN_quad, X_nl[test2], Y_nl[test2])

print("Coefficient of Determination — Nonlinear DGP")
print("="*72)
print("(Misspecified) Simple linear model on nonlinear data:")
print(f"  In-sample: R^2 = {metN_lin['R2']:.4f}, adj R^2 = {metN_lin['adj_R2']:.4f}")
print(f"  Out-of-sample R^2 (test): {r2_oos_N_lin:.4f}")
print("-"*48)
print("Transformed model with quadratic term:")
print(f"  In-sample: R^2 = {metN_quad['R2']:.4f}, adj R^2 = {metN_quad['adj_R2']:.4f}")
print(f"  Out-of-sample R^2 (test): {r2_oos_N_quad:.4f}")
print("="*72)

Coefficient of Determination — Linear DGP
In-sample: R^2 = 0.8544,  adj R^2 = 0.8543
Out-of-sample R^2 (test): 0.8357
------------------------------------------------------------------------
Coefficient of Determination — Nonlinear DGP
(Misspecified) Simple linear model on nonlinear data:
  In-sample: R^2 = 0.0165, adj R^2 = 0.0159
  Out-of-sample R^2 (test): 0.0073
------------------------------------------------
Transformed model with quadratic term:
  In-sample: R^2 = 0.8872, adj R^2 = 0.8870
  Out-of-sample R^2 (test): 0.8850


Transformations to a straight line

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import jarque_bera
from scipy import stats

# Helpers
# ----------------------------
def _nanmin_safe(a):
    a = np.asarray(a, float)
    a = a[np.isfinite(a)]
    return np.nan if a.size == 0 else np.min(a)

def fit_transform_params(arr, kind):
    """Learn per-transform params on TRAIN data (shift to ensure positivity; lambda for Box-Cox)."""
    arr = np.asarray(arr, float)
    if kind == "identity":
        return {"shift": 0.0, "lambda": None}
    if kind in ("log", "sqrt", "boxcox"):
        m = _nanmin_safe(arr)
        shift = 0.0 if not np.isfinite(m) or m > 0 else (-m + 1e-6)
        out = {"shift": float(shift), "lambda": None}
        if kind == "boxcox":
            # boxcox returns (transformed, lambda) when lmbda=None
            _, lam = stats.boxcox(arr + shift)
            out["lambda"] = float(lam)
        return out
    if kind == "inv":
        # tiny shift if zeros present
        return {"shift": 1e-6 if np.any(np.isclose(arr, 0.0, atol=1e-6)) else 0.0, "lambda": None}
    raise ValueError(kind)

def apply_transform(arr, kind, params):
    arr = np.asarray(arr, float)
    a = arr + params.get("shift", 0.0)
    if kind == "identity": return arr
    if kind == "log":      return np.log(a)
    if kind == "sqrt":     return np.sqrt(np.where(a < 0, np.nan, a))
    if kind == "inv":      return 1.0 / a
    if kind == "boxcox":   return stats.boxcox(a, lmbda=params["lambda"])
    raise ValueError(kind)

def invert_transform(vals, kind, params):
    vals = np.asarray(vals, float)
    s = params.get("shift", 0.0)
    if kind == "identity": return vals
    if kind == "log":      return np.exp(vals) - s
    if kind == "sqrt":     return (vals ** 2) - s
    if kind == "inv":      return (1.0 / vals) - s
    if kind == "boxcox":
        lam = params["lambda"]
        if np.isclose(lam, 0.0): return np.exp(vals) - s
        return np.power(lam * vals + 1.0, 1.0 / lam) - s
    raise ValueError(kind)

# Model + diagnostics
# ----------------------------
def fit_and_diagnose(y_t, x_t):
    mask = np.isfinite(y_t) & np.isfinite(x_t)
    y, x = y_t[mask], x_t[mask]
    X = sm.add_constant(x, has_constant="add")
    m = sm.OLS(y, X).fit()

    res, fit = m.resid, m.fittedvalues
    aic, bic, r2a = float(m.aic), float(m.bic), float(m.rsquared_adj)
    rmse_t = float(np.sqrt(np.nanmean(res**2)))

    jb_stat, jb_p, _, _ = jarque_bera(res)
    bp_stat, bp_p, _, _ = het_breuschpagan(res, sm.add_constant(fit))

    try:
        reset_p = float(sm.stats.linear_reset(m, power=2, use_f=True).pvalue)
    except Exception:
        reset_p = np.nan

    return m, dict(aic=aic, bic=bic, r2_adj=r2a, rmse_t=rmse_t,
                   jb_p=float(jb_p), bp_p=float(bp_p), reset_p=reset_p, nobs=int(m.nobs))

def cv_rmse_original(y, x, yk, xk, k=5, seed=42):
    y, x = np.asarray(y, float), np.asarray(x, float)
    mask = np.isfinite(y) & np.isfinite(x)
    y, x = y[mask], x[mask]
    n = len(y)
    if n < max(6, k + 1): return np.nan

    rng = np.random.default_rng(seed)
    idx = np.arange(n); rng.shuffle(idx)
    folds = np.array_split(idx, k)

    errs = []
    for i in range(k):
        te = folds[i]; tr = np.concatenate([folds[j] for j in range(k) if j != i])
        y_tr, x_tr = y[tr], x[tr]; y_te, x_te = y[te], x[te]

        py, px = fit_transform_params(y_tr, yk), fit_transform_params(x_tr, xk)
        yt, xt = apply_transform(y_tr, yk, py), apply_transform(x_tr, xk, px)

        Xtr = sm.add_constant(xt, has_constant="add")
        m = sm.OLS(yt, Xtr, missing="drop").fit()

        xte_t = apply_transform(x_te, xk, px)
        Xte = sm.add_constant(xte_t, has_constant="add")
        yhat_t = m.predict(Xte)
        yhat = invert_transform(yhat_t, yk, py)

        errs.append(float(np.sqrt(np.nanmean((y_te - yhat) ** 2))))
    return float(np.mean(errs))

def compare_transformations(X, Y, kfold=5, seed=42):
    specs = [
        ("Y|X",              "identity", "identity"),
        ("logY|X",           "log",      "identity"),
        ("Y|logX",           "identity", "log"),
        ("logY|logX",        "log",      "log"),
        ("sqrtY|X",          "sqrt",     "identity"),
        ("Y|sqrtX",          "identity", "sqrt"),
        ("invY|X",           "inv",      "identity"),
        ("Y|invX",           "identity", "inv"),
        ("boxcoxY|X",        "boxcox",   "identity"),
        ("boxcoxY|logX",     "boxcox",   "log"),
        ("boxcoxY|sqrtX",    "boxcox",   "sqrt"),
        ("boxcoxY|invX",     "boxcox",   "inv"),
    ]

    rows, models = [], {}
    X, Y = np.asarray(X, float).ravel(), np.asarray(Y, float).ravel()

    for name, yk, xk in specs:
        try:
            py = fit_transform_params(Y, yk)
            px = fit_transform_params(X, xk)
            y_t = apply_transform(Y, yk, py)
            x_t = apply_transform(X, xk, px)

            m, d = fit_and_diagnose(y_t, x_t)
            cv = cv_rmse_original(Y, X, yk, xk, k=kfold, seed=seed)

            rows.append({
                "model": name, "y_transform": yk, "x_transform": xk,
                "y_shift": py.get("shift", 0.0), "x_shift": px.get("shift", 0.0),
                "y_lambda": py.get("lambda", np.nan),
                "intercept": float(m.params[0]), "slope": float(m.params[1]),
                "adj_R2": d["r2_adj"], "RMSE_transformed": d["rmse_t"],
                "CV_RMSE_original": cv, "AIC": d["aic"], "BIC": d["bic"],
                "JB_p": d["jb_p"], "BP_p": d["bp_p"], "RESET_p": d["reset_p"],
                "nobs": d["nobs"],
            })
            models[name] = m
        except Exception as e:
            rows.append({"model": name, "y_transform": yk, "x_transform": xk, "error": str(e)})

    df = pd.DataFrame(rows).sort_values(
        by=["CV_RMSE_original", "BIC", "AIC"], ascending=[True, True, True],
        na_position="last"
    ).reset_index(drop=True)
    return df, models

if __name__ == "__main__":
    # Synthetic data where sqrt transform suits Y ~ sqrt(X)
    rng = np.random.default_rng(0)
    X = np.linspace(1, 100, 500)
    Y = 2.0 * np.sqrt(X) + rng.normal(0, 0.3, size=X.size)

    summary, models = compare_transformations(X, Y, kfold=5, seed=42)
    pd.set_option("display.float_format", lambda v: f"{v:,.6f}")
    print(summary.head(10).to_string(index=False))

/tmp/ipython-input-1352601626.py:53: RuntimeWarning: invalid value encountered in power
  return np.power(lam * vals + 1.0, 1.0 / lam) - s


        model y_transform x_transform  y_shift  x_shift  y_lambda  intercept      slope   adj_R2  RMSE_transformed  CV_RMSE_original           AIC           BIC     JB_p     BP_p  RESET_p  nobs
    logY|logX         log         log 0.000000 0.000000       NaN   0.693197   0.499772 0.995468          0.030066          0.303394 -2,081.412651 -2,072.983435 0.000000 0.000000 0.662048   500
      Y|sqrtX    identity        sqrt 0.000000 0.000000       NaN   0.017925   1.996135 0.995617          0.303950          0.303424    232.047023    240.476240 0.045807 0.344933 0.623962   500
boxcoxY|sqrtX      boxcox        sqrt 0.000000 0.000000  1.426591  -9.619935   5.731571 0.987176          1.499153          0.548640  1,827.838772  1,836.267989 0.000071 0.000012 0.000000   500
    boxcoxY|X      boxcox    identity 0.000000 0.000000  1.426591   5.724549   0.459390 0.985477          1.595349          0.640978  1,890.030814  1,898.460030 0.000000 0.000121 0.000000   500
          Y|X    identity    i

Correlation (Pearson \& Spearman) and its link to $R^2$

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
from scipy import stats
import statsmodels.api as sm

rng = np.random.default_rng(42)

# 1) Linear relation: verify r^2 = R^2
# ----------------------------
n = 1000
beta0, beta1 = 0.5, 1.3
X = rng.normal(0.0, 1.0, n)
eps = rng.normal(0.0, 0.6, n)
Y = beta0 + beta1 * X + eps

r_pearson, pval = stats.pearsonr(X, Y)
r2 = r_pearson**2

X1 = sm.add_constant(X, has_constant="add")
ols = sm.OLS(Y, X1).fit()
R2 = float(ols.rsquared)

print("="*72)
print("Pearson correlation and R^2 in simple linear regression")
print("="*72)
print(f"r (Pearson)   = {r_pearson:.4f}")
print(f"r^2           = {r2:.4f}")
print(f"R^2 (OLS)     = {R2:.4f}   <-- should match r^2 up to rounding")
print("-"*72)

# 2) Outlier sensitivity: Pearson vs Spearman
# ----------------------------
X_out = np.append(X, 4.0)
Y_out = np.append(Y, 30.0)  # extreme outlier
r_p_out, _ = stats.pearsonr(X_out, Y_out)
r_s_out, _ = stats.spearmanr(X_out, Y_out)

print("Outlier sensitivity")
print(f"  Pearson r (no outlier)   = {r_pearson:.4f}")
print(f"  Pearson r (with outlier) = {r_p_out:.4f}   <-- can shift notably")
print(f"  Spearman rho (with outlier) = {r_s_out:.4f} (rank-based, more robust)")
print("-"*72)

# 3) Non-linear but monotonic: Spearman captures it better
#    Y = exp(X) + noise  => monotonic non-linear
# ----------------------------
X_mono = rng.normal(0.0, 1.0, n)
Y_mono = np.exp(0.8 * X_mono) + rng.normal(0.0, 0.2, n)

r_p_mono, _ = stats.pearsonr(X_mono, Y_mono)
r_s_mono, _ = stats.spearmanr(X_mono, Y_mono)

print("Monotonic non-linear relationship")
print(f"  Pearson r  = {r_p_mono:.4f}   (may understate strength)")
print(f"  Spearman ρ = {r_s_mono:.4f}   (captures monotonic link)")
print("-"*72)

# 4) Spurious correlation in random walks (non-stationary levels)
# ----------------------------
T = 1500
w1 = np.cumsum(rng.normal(0.0, 1.0, T))  # RW1 (price-like)
w2 = np.cumsum(rng.normal(0.0, 1.0, T))  # RW2 (independent)

r_levels, _ = stats.pearsonr(w1, w2)   # often |r| noticeably > 0 spuriously
r_diff, _   = stats.pearsonr(np.diff(w1), np.diff(w2))  # near 0 when differenced

print("Spurious correlation in non-stationary series")
print(f"  Pearson r (levels)     = {r_levels:.4f}   (can be misleadingly high)")
print(f"  Pearson r (differences)= {r_diff:.4f}   (near zero for independent increments)")
print("="*72)

Pearson correlation and R^2 in simple linear regression
r (Pearson)   = 0.9032
r^2           = 0.8158
R^2 (OLS)     = 0.8158   <-- should match r^2 up to rounding
------------------------------------------------------------------------
Outlier sensitivity
  Pearson r (no outlier)   = 0.9032
  Pearson r (with outlier) = 0.8181   <-- can shift notably
  Spearman rho (with outlier) = 0.8926 (rank-based, more robust)
------------------------------------------------------------------------
Monotonic non-linear relationship
  Pearson r  = 0.8506   (may understate strength)
  Spearman ρ = 0.9625   (captures monotonic link)
------------------------------------------------------------------------
Spurious correlation in non-stationary series
  Pearson r (levels)     = -0.7369   (can be misleadingly high)
  Pearson r (differences)= -0.0226   (near zero for independent increments)


Isolating true alpha from systematic betas

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
import statsmodels.api as sm
from scipy import stats

# Helpers
def _nw_lags(n: int) -> int:
    """Automatic Newey–West lag: floor(1.5 * n^(1/3)); at least 1."""
    return max(1, int(np.floor(1.5 * (n ** (1/3)))))

def annualize_mean(mu: float, periods_per_year: int, compounding: bool = False) -> float:
    """Annualize a per-period mean return."""
    if compounding:
        return (1.0 + mu) ** periods_per_year - 1.0
    return periods_per_year * mu

def newey_west_alpha_test(
    df: pd.DataFrame,
    factors: list[str],
    ret_col: str = "ret",
    rf_col: str = "rf",
    periods_per_year: int = 252,
    alpha_sig: float = 0.05,
    hurdle_annual: float = 0.0,
    compound_annualization: bool = False,
) -> dict:
    """Estimate alpha via OLS with Newey–West (HAC) SEs, one-sided test H0: alpha = 0,
    and compute the one-sided lower confidence bound. Decide deploy if lower_annual > hurdle_annual.
    """
    # 1) Prepare y (excess returns) and X (factors + constant)
    cols = [ret_col, rf_col] + factors
    data = df[cols].dropna()
    y = data[ret_col] - data[rf_col]
    X = sm.add_constant(data[factors], has_constant="add")

    # 2) Fit OLS with HAC covariance (Newey–West) and automatic lag choice
    n = int(y.shape[0])
    lags = _nw_lags(n)
    model = sm.OLS(y.values, X.values)
    res = model.fit(cov_type="HAC", cov_kwds={"maxlags": lags})

    # 3) Extract alpha (intercept) and HAC inference
    alpha_hat = float(res.params[0])   # const is first column after add_constant
    se_alpha  = float(res.bse[0])
    t_alpha   = float(res.tvalues[0])
    df_resid  = int(res.df_resid)

    # 4) One-sided p-value for H_a: alpha > 0
    p_one_sided = 1.0 - stats.t.cdf(t_alpha, df_resid)

    # 5) One-sided lower (1 - alpha_sig) bound for alpha
    tcrit = stats.t.ppf(1.0 - alpha_sig, df_resid)
    lower_alpha = alpha_hat - tcrit * se_alpha

    # 6) Annualize alpha and compare to hurdle
    alpha_annual = annualize_mean(alpha_hat, periods_per_year, compound_annualization)
    lower_alpha_annual = annualize_mean(lower_alpha, periods_per_year, compound_annualization)
    decision_deploy = bool(lower_alpha_annual > hurdle_annual)

    # 7) Package results (betas with HAC t-stats)
    betas = pd.Series(res.params[1:], index=factors, name="beta")
    betas_se = pd.Series(res.bse[1:], index=factors, name="se")
    betas_t  = pd.Series(res.tvalues[1:], index=factors, name="t")

    return {
        "n_obs": n,
        "nw_lags": lags,
        "alpha_per_period": alpha_hat,
        "alpha_se": se_alpha,
        "alpha_t": t_alpha,
        "alpha_p_one_sided": p_one_sided,
        "alpha_lower_bound_per_period": lower_alpha,
        "alpha_annual": alpha_annual,
        "alpha_lower_bound_annual": lower_alpha_annual,
        "hurdle_annual": hurdle_annual,
        "deploy_if_lower_gt_hurdle": decision_deploy,
        "rsquared": float(res.rsquared),
        "adj_rsquared": float(res.rsquared_adj),
        "betas": pd.concat([betas, betas_se, betas_t], axis=1),  # columns: beta, se, t
    }

# Synthetic data creation
def _ar1(mu: float, phi: float, sigma: float, size: int, rng: np.random.Generator) -> np.ndarray:
    """AR(1) generator used for factors."""
    e = rng.normal(0.0, sigma, size)
    x = np.empty(size, dtype=float)
    x[0] = mu
    for t in range(1, size):
        x[t] = mu + phi * (x[t - 1] - mu) + e[t]
    return x

def create_synthetic_df(
    T: int = 2000,
    seed: int = 42,
    periods_per_year: int = 252,
    alpha_annual_true: float = 0.04,
) -> tuple[pd.DataFrame, dict]:
    """Create DataFrame `df` with columns: ret, rf, MKT, SMB, HML, MOM."""
    rng = np.random.default_rng(seed)

    # Risk-free (constant daily)
    rf_annual = 0.015
    rf = np.full(T, rf_annual / periods_per_year, dtype=float)

    # Factors: AR(1) with small positive drifts
    MKT = _ar1(mu=0.00040, phi=0.20, sigma=0.010, size=T, rng=rng)  # ~10% annual mean
    SMB = _ar1(mu=0.00010, phi=0.15, sigma=0.006, size=T, rng=rng)
    HML = _ar1(mu=0.00010, phi=0.10, sigma=0.006, size=T, rng=rng)
    MOM = _ar1(mu=0.00015, phi=0.15, sigma=0.008, size=T, rng=rng)

    # True parameters
    alpha_pp = alpha_annual_true / periods_per_year
    betas_true = np.array([0.80, 0.30, 0.20, -0.10])  # [MKT, SMB, HML, MOM]

    # Residuals: AR(1) with heteroskedastic scale depending on |MKT|
    rho = 0.40
    sigma_eps = 0.006
    eps = np.empty(T, dtype=float)
    eps[0] = rng.normal(0.0, sigma_eps)
    for t in range(1, T):
        scale_t = sigma_eps * (0.5 + 0.5 * abs(MKT[t]))  # state-dependent variance
        eps[t] = rho * eps[t - 1] + rng.normal(0.0, scale_t)

    # Strategy returns: ret = rf + alpha + factors @ beta + noise
    factors_mat = np.column_stack([MKT, SMB, HML, MOM])
    ret = rf + alpha_pp + factors_mat @ betas_true + eps

    dates = pd.date_range("2015-01-01", periods=T, freq="B")
    df = pd.DataFrame(
        {"ret": ret, "rf": rf, "MKT": MKT, "SMB": SMB, "HML": HML, "MOM": MOM},
        index=dates,
    )

    truth = {
        "true_alpha_annual": alpha_annual_true,
        "true_betas": pd.Series(betas_true, index=["MKT", "SMB", "HML", "MOM"]),
        "periods_per_year": periods_per_year,
    }
    return df, truth

if __name__ == "__main__":
    # Config
    T = 2000
    PPY = 252
    HURDLE = 0.02  # 2% annual lower-bound hurdle
    SEED = 42

    # Create synthetic DataFrame `df`
    df, truth = create_synthetic_df(T=T, seed=SEED, periods_per_year=PPY, alpha_annual_true=0.04)
    factors = ["MKT", "SMB", "HML", "MOM"]

    # Run Newey–West alpha test
    out = newey_west_alpha_test(
        df,
        factors=factors,
        periods_per_year=PPY,
        hurdle_annual=HURDLE,
        alpha_sig=0.05,  # one-sided 95% lower bound
    )

    # Pretty print summary
    def _fmt_pct(x: float) -> str:
        return f"{100 * x:.2f}%"

    print("=" * 72)
    print("Newey–West Factor Alpha Test (synthetic data)")
    print("=" * 72)
    print(f"Observations:       {out['n_obs']}")
    print(f"Newey–West lags:    {out['nw_lags']}")
    print(f"R^2 / Adj R^2:      {out['rsquared']:.4f} / {out['adj_rsquared']:.4f}")
    print("-" * 72)
    print(f"True alpha (annual):        {_fmt_pct(truth['true_alpha_annual'])}")
    print(f"Estimated alpha (annual):   {_fmt_pct(out['alpha_annual'])}")
    print(f"Lower bound (annual):       {_fmt_pct(out['alpha_lower_bound_annual'])}")
    print(f"Hurdle (annual):            {_fmt_pct(out['hurdle_annual'])}")
    print(f"Deploy?                     {out['deploy_if_lower_gt_hurdle']}")
    print("-" * 72)
    print("Alpha (per period):")
    print(f"  alpha_hat   = {out['alpha_per_period']:.6f}")
    print(f"  se(alpha)   = {out['alpha_se']:.6f}")
    print(f"  t(alpha)    = {out['alpha_t']:.3f}")
    print(f"  p_one_sided = {out['alpha_p_one_sided']:.4g}")
    print("-" * 72)

    # Betas table (with true betas for reference)
    betas_tbl = out["betas"].copy()
    betas_tbl["beta_true"] = truth["true_betas"]
    betas_tbl = betas_tbl[["beta", "se", "t", "beta_true"]]
    print("Factor loadings (HAC):")
    print(betas_tbl.to_string(float_format=lambda x: f"{x: .4f}"))
    print("=" * 72)

Newey–West Factor Alpha Test (synthetic data)
Observations:       2000
Newey–West lags:    18
R^2 / Adj R^2:      0.8631 / 0.8628
------------------------------------------------------------------------
True alpha (annual):        4.00%
Estimated alpha (annual):   2.52%
Lower bound (annual):       -2.23%
Hurdle (annual):            2.00%
Deploy?                     False
------------------------------------------------------------------------
Alpha (per period):
  alpha_hat   = 0.000100
  se(alpha)   = 0.000114
  t(alpha)    = 0.873
  p_one_sided = 0.1915
------------------------------------------------------------------------
Factor loadings (HAC):
       beta      se         t  beta_true
MKT  0.7900  0.0075  105.9068     0.8000
SMB  0.2785  0.0142   19.6338     0.3000
HML  0.2018  0.0123   16.4462     0.2000
MOM -0.1038  0.0096  -10.8553    -0.1000


Neutralizing unintended factor betas (minimum-variance hedge)

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
import statsmodels.api as sm

# Utilities
# ----------------------------
def ols_beta(y: np.ndarray, X: np.ndarray) -> np.ndarray:
    """OLS coefficients with intercept. Returns [alpha, beta_1,...,beta_k]."""
    Xc = sm.add_constant(X, has_constant="add")
    return sm.OLS(y, Xc).fit().params

def minvar_hedge_weights(beta_S: np.ndarray,
                         B_S: np.ndarray,
                         Sigma: np.ndarray) -> np.ndarray:
    """
    Minimum-variance hedge:
        minimize w' Sigma w   s.t.   B_S' w = -beta_S
    Closed-form: w* = Sigma^{-1} B_S (B_S' Sigma^{-1} B_S)^{-1} (-beta_S)
    Shapes:
      beta_S: (m,)           m = #factors to neutralize
      B_S:    (n, m)         n = #hedge instruments, rows= instruments, cols= factors S
      Sigma:  (n, n)         covariance of hedge instruments' returns
    Returns w*: (n,)
    """
    # numerical guards
    I = np.eye(Sigma.shape[0])
    Sinv = np.linalg.inv(Sigma + 1e-10 * I)
    M = B_S.T @ Sinv @ B_S
    M_inv = np.linalg.inv(M + 1e-10 * np.eye(M.shape[0]))
    w = Sinv @ B_S @ (M_inv @ (-beta_S))
    return w

# Synthetic data
# ----------------------------
rng = np.random.default_rng(7)
T = 1500
K = 3  # factors: MKT, SMB, HML
dates = pd.date_range("2018-01-01", periods=T, freq="B")

# factors (AR(1) with drift)
def ar1(mu, phi, sigma, size):
    e = rng.normal(0.0, sigma, size)
    x = np.empty(size)
    x[0] = mu
    for t in range(1, size):
        x[t] = mu + phi * (x[t-1] - mu) + e[t]
    return x

MKT = ar1(0.0003, 0.2, 0.01, T)
SMB = ar1(0.0001, 0.2, 0.007, T)
HML = ar1(0.0001, 0.2, 0.007, T)
F = np.column_stack([MKT, SMB, HML])

# strategy with unintended betas (needs hedging)
alpha_pp = 0.04 / 252
beta_true_strat = np.array([0.65, 0.35, 0.20])   # MKT, SMB, HML
eps_s = rng.normal(0, 0.006, T)
ret_strat = alpha_pp + F @ beta_true_strat + eps_s

# hedge instruments: ES (MKT), IWM (SMB), VLUE (HML) with noisy exposures
B_hedge_true = np.array([
    [ 1.05,  0.05, -0.02],  # ES ~ market future
    [ 0.20,  0.95,  0.05],  # IWM ~ small caps
    [ 0.10,  0.05,  1.00],  # VLUE ~ value
])  # shape (3 instruments × 3 factors)
noise = rng.normal(0, 0.004, size=(T, 3))
ret_hedge = F @ B_hedge_true.T + noise  # (T × 3)

# pack DataFrames
df = pd.DataFrame(F, index=dates, columns=["MKT","SMB","HML"])
df["RET_STRAT"] = ret_strat
df_hedge = pd.DataFrame(ret_hedge, index=dates, columns=["ES","IWM","VLUE"])

# Estimate betas by OLS
# strategy betas
# ----------------------------
params_strat = ols_beta(df["RET_STRAT"].values, df[["MKT","SMB","HML"]].values)
alpha_hat, beta_hat_strat = params_strat[0], params_strat[1:]

# Hedge instrument betas (each instrument on the same factors)
B_hat = []
for col in df_hedge.columns:
    params = ols_beta(df_hedge[col].values, df[["MKT","SMB","HML"]].values)
    B_hat.append(params[1:])  # skip intercept -> only factor loadings
B_hat = np.vstack(B_hat)  # shape (n_instruments × K)

# Choose factors to neutralize and compute weights
# ----------------------------
factors = ["MKT","SMB","HML"]
neutralize = ["MKT","SMB","HML"]  # customize subset if desired (e.g., ["MKT","SMB"])
idx = [factors.index(f) for f in neutralize]

beta_S = beta_hat_strat[idx]           # (m,)
B_S    = B_hat[:, idx]                 # (n × m)
Sigma  = np.cov(df_hedge.values, rowvar=False)  # (n × n)

w_star = minvar_hedge_weights(beta_S, B_S, Sigma)  # hedge weights per unit strategy notional

# Verify exposures before/after
# ----------------------------
pre_expo  = beta_hat_strat
post_expo = beta_hat_strat + B_hat.T @ w_star  # new betas after overlay

print("Estimated strategy betas (pre-hedge):", dict(zip(factors, pre_expo.round(3))))
print("Estimated hedge weights (ES, IWM, VLUE):", np.round(w_star, 4))
print("Estimated strategy betas (post-hedge):", dict(zip(factors, post_expo.round(3))))

Estimated strategy betas (pre-hedge): {'MKT': np.float64(0.627), 'SMB': np.float64(0.372), 'HML': np.float64(0.191)}
Estimated hedge weights (ES, IWM, VLUE): [-0.5145 -0.3535 -0.1575]
Estimated strategy betas (post-hedge): {'MKT': np.float64(0.0), 'SMB': np.float64(0.0), 'HML': np.float64(0.0)}


Orthogonalizing a signal against factors (Frisch–Waugh–Lovell)

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
import statsmodels.api as sm
from scipy import stats

# Helpers
# ----------------------------
def _nw_lags(n: int) -> int:
    return max(1, int(np.floor(1.5 * (n ** (1/3)))))

def zscore(a: pd.Series) -> pd.Series:
    return (a - a.mean()) / a.std(ddof=1)

def hac_fit(y: pd.Series, X: pd.DataFrame):
    n = len(y)
    lags = _nw_lags(n)
    res = sm.OLS(y.values, sm.add_constant(X.values, has_constant="add")).fit(
        cov_type="HAC", cov_kwds={"maxlags": lags}
    )
    return res, lags

def ar1(mu, phi, sigma, size, rng):
    e = rng.normal(0.0, sigma, size)
    x = np.empty(size)
    x[0] = mu
    for t in range(1, size):
        x[t] = mu + phi * (x[t-1] - mu) + e[t]
    return x


# Synthetic data
# ----------------------------
rng = np.random.default_rng(123)
T, PPY = 2000, 252
dates = pd.date_range("2015-01-01", periods=T, freq="B")

# Factor and latent edge
MKT = ar1(0.00030, 0.20, 0.010, T, rng)      # dominant factor
EDGE = ar1(0.00000, 0.35, 0.008, T, rng)     # latent true edge (independent)

# Raw signal contaminated by MKT
raw_signal = 0.7 * EDGE + 0.6 * MKT + rng.normal(0, 0.005, T)

# Returns driven by MKT + lagged EDGE (tradable edge) + noise
alpha_pp = 0.05 / PPY                         # ~5% annual edge strength
ret_excess = 0.6 * MKT + alpha_pp * zscore(pd.Series(EDGE)).shift(1).fillna(0).values \
             + rng.normal(0, 0.006, T)

rf = np.full(T, 0.015 / PPY)                  # constant risk-free (not used here directly)

df = pd.DataFrame({
    "RET_XS": ret_excess,  # excess returns
    "MKT": MKT,
    "signal_raw": raw_signal,
}, index=dates)

# Orthogonalize signal against factors (FWL residualization)
# ----------------------------
factors = ["MKT"]  # extend as needed, e.g., ["MKT","SMB","HML","MOM"]
# regress signal_raw on factors (contemporaneous) -> residuals
res_sig = sm.OLS(df["signal_raw"].values,
                 sm.add_constant(df[factors].values, has_constant="add")).fit()
signal_ortho = pd.Series(res_sig.resid, index=df.index, name="signal_ortho")

# Standardize and apply a one-period trade lag to avoid lookahead
s_raw_lag   = zscore(df["signal_raw"]).shift(1)
s_ortho_lag = zscore(signal_ortho).shift(1)

# Prepare regression datasets (drop NaNs from lagging)
data_base = pd.concat([df["RET_XS"], df[factors], s_raw_lag.rename("S")], axis=1).dropna()
data_ortho = pd.concat([df["RET_XS"], df[factors], s_ortho_lag.rename("S")], axis=1).dropna()


# HAC (Newey–West) regressions
#   RET_XS ~ const + factors + S
# ----------------------------
res_base, lags_base = hac_fit(data_base["RET_XS"], data_base[factors + ["S"]])
res_ortho, lags_ortho = hac_fit(data_ortho["RET_XS"], data_ortho[factors + ["S"]])

# Extract signal coefficients (assumes order: const, factors..., S)
coef_S_base = float(res_base.params[-1]);  t_S_base = float(res_base.tvalues[-1])
coef_S_orth = float(res_ortho.params[-1]); t_S_orth = float(res_ortho.tvalues[-1])

# Diagnostics: correlation of tradable signal with MKT (should drop to ~0 after ortho)
corr_raw = float(np.corrcoef(s_raw_lag.dropna(), data_base["MKT"])[0,1])
corr_ort = float(np.corrcoef(s_ortho_lag.dropna(), data_ortho["MKT"])[0,1])

# One-sided p-values for H_a: beta_S > 0
df_b = int(res_base.df_resid);  p_raw = 1.0 - stats.t.cdf(t_S_base, df_b)
df_o = int(res_ortho.df_resid); p_ort = 1.0 - stats.t.cdf(t_S_orth, df_o)

print("="*72)
print("Refining Core Logic via Signal Orthogonalization (synthetic demo)")
print("="*72)
print(f"NW lags (base / ortho): {lags_base} / {lags_ortho}")
print(f"Corr(S_raw_lag, MKT):   {corr_raw: .4f}")
print(f"Corr(S_ortho_lag, MKT): {corr_ort: .4f}")
print("-"*72)
print("RET_XS ~ const + MKT + S")
print(f"  Raw S   : beta={coef_S_base: .5f},  t_NW={t_S_base: .3f},  p_one_sided={p_raw:.4g}")
print(f"  Ortho S : beta={coef_S_orth: .5f},  t_NW={t_S_orth: .3f},  p_one_sided={p_ort:.4g}")
print("-"*72)
print(f"R^2 (base / ortho): {res_base.rsquared:.4f} / {res_ortho.rsquared:.4f}")
print("="*72)

Refining Core Logic via Signal Orthogonalization (synthetic demo)
NW lags (base / ortho): 18 / 18
Corr(S_raw_lag, MKT):    0.1193
Corr(S_ortho_lag, MKT): -0.0208
------------------------------------------------------------------------
RET_XS ~ const + MKT + S
  Raw S   : beta=-0.00003,  t_NW=-0.207,  p_one_sided=0.582
  Ortho S : beta= 0.00007,  t_NW= 0.570,  p_one_sided=0.2844
------------------------------------------------------------------------
R^2 (base / ortho): 0.5052 / 0.5052


Factor-aware multi-strategy allocation

In [ ]:
from __future__ import annotations
import numpy as np, pandas as pd
import statsmodels.api as sm

# Helpers
# ----------------------------
def ols_alpha_beta(y: np.ndarray, X: np.ndarray):
    """Return (alpha, betas, resid) from OLS of y on [const, X]."""
    Xc = sm.add_constant(X, has_constant="add")
    res = sm.OLS(y, Xc).fit()
    alpha = float(res.params[0])
    betas = res.params[1:].astype(float)
    resid = y - res.fittedvalues
    return alpha, betas, resid

def nullspace(A: np.ndarray, atol: float = 1e-12, rtol: float = 0.0) -> np.ndarray:
    """Right-nullspace of A using SVD; columns form an orthonormal basis."""
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    tol = max(atol, rtol * s[0]) if s.size > 0 else atol
    nnz = (s >= tol).sum()
    return vh[nnz:].T  # shape: (ncols(A) x nullity)

def project_to_nullspace(w: np.ndarray, A: np.ndarray) -> np.ndarray:
    """Orthogonal projection of vector w onto Null(A)."""
    N = nullspace(A)
    if N.size == 0:  # no degrees of freedom; return zero vector
        return np.zeros_like(w)
    return N @ (N.T @ w)  # since columns of N are orthonormal

def annualize_mean(mu: float, ppy: int) -> float:
    return mu * ppy

def annualize_vol(sig: float, ppy: int) -> float:
    return sig * np.sqrt(ppy)

# Synthetic data: 4 strategies, 3 factors (MKT, SMB, HML)
# ----------------------------
rng = np.random.default_rng(11)
T, PPY = 2500, 252
dates = pd.date_range("2014-01-01", periods=T, freq="B")

def ar1(mu, phi, sigma, n):
    e = rng.normal(0.0, sigma, n)
    x = np.empty(n)
    x[0] = mu
    for t in range(1, n):
        x[t] = mu + phi * (x[t-1] - mu) + e[t]
    return x

# Factors
MKT = ar1(0.00035, 0.20, 0.010, T)
SMB = ar1(0.00010, 0.25, 0.007, T)
HML = ar1(0.00010, 0.15, 0.006, T)
F   = np.column_stack([MKT, SMB, HML])

# True strategy alphas (annual) and betas (rows=strategies, cols=factors)
alpha_ann_true = np.array([0.06, 0.04, 0.03, 0.05])
alpha_pp_true  = alpha_ann_true / PPY
B_true = np.array([
    [ 0.80,  0.35,  0.10],  # S1: strong MKT, SMB tilt
    [ 0.70,  0.30,  0.05],  # S2: similar hidden betas
    [ 0.40,  0.15,  0.30],  # S3: more value exposure
    [ 0.75,  0.25,  0.00],  # S4: market-heavy
])  # shape (4 x 3)

# Idiosyncratic noises (mild AR(1)) per strategy
rho = np.array([0.30, 0.35, 0.25, 0.20])
sig = np.array([0.007, 0.006, 0.006, 0.007])
E = np.zeros((T, 4))
E[0] = rng.normal(0, sig, 4)
for t in range(1, T):
    E[t] = rho * E[t-1] + rng.normal(0, sig, 4)

# Strategy returns: r_i = alpha_i + F @ beta_i + eps_i   (use excess returns)
R = (F @ B_true.T) + alpha_pp_true + E  # (T x 4)
df = pd.DataFrame(R, index=dates, columns=["S1","S2","S3","S4"])
df_f = pd.DataFrame(F, index=dates, columns=["MKT","SMB","HML"])


# Estimate betas and residuals for each strategy
# ----------------------------
alphas_hat, betas_hat, residuals = [], [], []
for s in df.columns:
    a, b, e = ols_alpha_beta(df[s].values, df_f.values)
    alphas_hat.append(a); betas_hat.append(b); residuals.append(e)

alphas_hat = np.array(alphas_hat)                 # (4,)
B_hat      = np.vstack(betas_hat)                 # (4 x 3)
E_hat      = np.column_stack(residuals)           # (T x 4)
mu_resid   = E_hat.mean(axis=0)                   # (4,)
Sigma_resid = np.cov(E_hat, rowvar=False)         # (4 x 4)


# Build allocations
# 1) Naive equal-weight (EW)
# 2) Factor-aware: neutral to {MKT, SMB} and maximize residual Sharpe
#    Unconstrained MV-SR direction: w_u ~ Σ^{-1} mu_resid
#    Project w_u onto Null(B_subset' ) to enforce factor-neutrality
# ----------------------------
n_strats = B_hat.shape[0]
EW = np.full(n_strats, 1.0 / n_strats)

# Unconstrained direction (residual maximum Sharpe)
w_u = np.linalg.solve(Sigma_resid + 1e-10 * np.eye(n_strats), mu_resid)

# Factor subset to neutralize
factors = ["MKT","SMB","HML"]
subset  = ["MKT","SMB"]         # make portfolio neutral to MKT & SMB
idx = [factors.index(f) for f in subset]
A = B_hat[:, idx].T             # constraints: A @ w = 0  (portfolio exposures)

# Project onto Null(A)
w_p = project_to_nullspace(w_u, A)

# Normalize (net = 1 if possible; otherwise gross = 1)
if abs(w_p.sum()) > 1e-8:
    w_p = w_p / w_p.sum()
else:
    w_p = w_p / (np.abs(w_p).sum() + 1e-12)

# Diagnostics and comparison
# ----------------------------
def port_metrics(w):
    expo = B_hat.T @ w                         # factor exposures
    mu_p = float(mu_resid @ w)                 # residual mean (per period)
    sig_p = float(np.sqrt(w @ (Sigma_resid @ w)))  # residual vol (per period)
    ann_mu  = annualize_mean(mu_p, PPY)
    ann_vol = annualize_vol(sig_p, PPY)
    ann_sr  = ann_mu / (ann_vol + 1e-12)
    alpha_ann_est = annualize_mean(float(alphas_hat @ w), PPY)
    return {"w": w, "expo": expo, "mu_resid_ann": ann_mu, "vol_resid_ann": ann_vol,
            "SR_resid_ann": ann_sr, "alpha_ann_est": alpha_ann_est}

m_ew = port_metrics(EW)
m_p  = port_metrics(w_p)

# Print results
def fmt(x):
    return np.array2string(x, precision=3, floatmode="fixed")

print("="*72)
print("Factor-aware multi-strategy allocation (synthetic)")
print("="*72)
print("Equal-Weight portfolio:")
print("  weights        :", fmt(m_ew["w"]))
print("  exposures (MKT, SMB, HML):", fmt(m_ew["expo"]))
print(f"  residual mu/vol/SR (ann): {m_ew['mu_resid_ann']:.2%} / {m_ew['vol_resid_ann']:.2%} / {m_ew['SR_resid_ann']:.2f}")
print(f"  alpha_est_annual: {m_ew['alpha_ann_est']:.2%}")
print("-"*72)
print("Factor-neutral (MKT & SMB) residual-optimized portfolio:")
print("  weights        :", fmt(m_p["w"]))
print("  exposures (MKT, SMB, HML):", fmt(m_p["expo"]))
print(f"  residual mu/vol/SR (ann): {m_p['mu_resid_ann']:.2%} / {m_p['vol_resid_ann']:.2%} / {m_p['SR_resid_ann']:.2f}")
print(f"  alpha_est_annual: {m_p['alpha_ann_est']:.2%}")
print("="*72)

Factor-aware multi-strategy allocation (synthetic)
Equal-Weight portfolio:
  weights        : [0.250 0.250 0.250 0.250]
  exposures (MKT, SMB, HML): [0.656 0.251 0.135]
  residual mu/vol/SR (ann): -0.00% / 5.34% / -0.00
  alpha_est_annual: 2.36%
------------------------------------------------------------------------
Factor-neutral (MKT & SMB) residual-optimized portfolio:
  weights        : [ 0.142 -0.081 -0.039 -0.056]
  exposures (MKT, SMB, HML): [-4.857e-17 -1.388e-17 -2.665e-05]
  residual mu/vol/SR (ann): 0.00% / 1.97% / 0.00
  alpha_est_annual: 1.91%
